# Code to process the wikidata provided by Wenhao

vol1/data/WIKIPEDIA/kg/

parsed_page.jsonl {"page_id": "12", "item_id": "6199", "title": "Anarchism"}

page.csv: page_id,item_id,title,views

statements.csv: source_item_id,edge_property_id,target_item_id

item.csv: item_id,en_label,en_description

property.csv: property_id,en_label,en_description


In [1]:
import os 
path = "/Users/mengxiayu/Documents/Research/WIKIPEDIA/kg/"
import pandas as pd
df_item = pd.read_csv(path+"item.csv")
print("df_item size", len(df_item))

df_statement = pd.read_csv(path + "statements.csv")
print("df_statement size", len(df_statement))

df_property = pd.read_csv(path + "property.csv")
print("df_property size", len(df_property))


df_item size 51450316
df_statement size 141206853
df_property size 6985


In [2]:
def item2name(item):
    return df_item.loc[df_item["item_id"] == item]["en_label"].to_string(index=False)
def prop2name(property):
    return df_property.loc[df_property["property_id"] == property]["en_label"].to_string(index=False)

the distribution of properties and values

In [3]:
from collections import Counter
cnt_properties = Counter(df_statement["edge_property_id"])

In [4]:
for k,v in cnt_properties.most_common(20):
    print(prop2name(k))

instance of
published in
country
language of work or name
located in the administrative territorial entity
main subject
occupation
country of citizenship
given name
taxon rank
place of birth
located in time zone
parent taxon
subclass of
family name
sex or gender
sport
member of sports team
location
educated at


“instance of” can be used for classify item.
edge_property_id: 31 

In [5]:
selected = df_statement.loc[df_statement["edge_property_id"] == 31]
cnt_values = Counter(selected["target_item_id"])

most frequent property values

In [6]:
for prop, freq in cnt_values.most_common(20):
    print(freq, prop, item2name(prop))

6221695 5 human
2617347 16521 taxon
1186867 7187 gene
973318 8054 protein
588507 13100073 village-level division in China
524700 8502 mountain
456658 486972 human settlement
415212 523 star
397810 4022 river
379630 79007 street
322701 54050 hill
321738 30612 clinical trial
301216 101352 family name
281507 318 galaxy
273055 532 village
262893 11173 chemical compound
246901 3863 asteroid
245900 23397 lake
237043 11424 film
215391 482994 album


most frequent properties of human.

**human items has 467 distinct properties**

In [7]:
items_human = df_statement.loc[(df_statement["edge_property_id"] == 31) & (df_statement["target_item_id"] == 5)]["source_item_id"] 
print("length of items_human", len(items_human))
selected_statements = df_statement.loc[df_statement["source_item_id"].isin(items_human)] 
print("length of human statements", len(selected_statements))
cnt = Counter(selected_statements["edge_property_id"])
for k,v in cnt.most_common(20):
    print(v, k, prop2name(k))
print(len(set(selected_statements["edge_property_id"])))

length of items_human 6221695
length of human statements 34447385
6224170 31 instance of
4284685 106 occupation
3491940 27 country of citizenship
2778825 735 given name
2242227 19 place of birth
1686497 734 family name
1657609 21 sex or gender
1318256 54 member of sports team
1165914 69 educated at
856722 20 place of death
846451 1412 languages spoken, written or signed
841587 641 sport
697273 108 employer
660652 166 award received
563212 39 position held
405978 1344 participant of
393673 102 member of political party
333690 413 position played on team / speciality
254962 937 work location
222758 1343 described by source
467


look at occupation value

**property "occupation" has 8505 distinct values**

In [8]:
cnt_occupation = Counter(selected_statements.loc[selected_statements["edge_property_id"] == 106]["target_item_id"])
for k,v in cnt_occupation.most_common(10):
    print(v, k, item2name(k))
print(len(set(cnt_occupation)))

597418 82955 politician
256208 937857 association football player
246150 36180 writer
216224 33999 actor
115786 1930187 journalist
78314 49757 poet
77140 36834 composer
72524 47064 military personnel
67334 40348 lawyer
55790 43845 businessperson
8505


most frequent country of citizenship. USA: 400,396

In [9]:
cnt_country = Counter(selected_statements.loc[selected_statements["edge_property_id"]==27]["target_item_id"])
print("# values of 'country of citizenship'", len(cnt_country))
for k,v in cnt_country.most_common(10):
    print(v, k, item2name(k))


# values of 'country of citizenship' 2484
400396 30 United States of America
234488 183 Germany
218384 142 France
160015 17 Japan
135986 145 United Kingdom
124611 9903 Ming dynasty
111262 29 Spain
86070 15180 Soviet Union
86046 38 Italy
75137 16 Canada


count the amount of USA citizens

In [10]:
item_usa_citizen = selected_statements.loc[(selected_statements["edge_property_id"]==27) & (selected_statements["target_item_id"]==30)]["source_item_id"]
print(len(item_usa_citizen))

400396


create statement df for usa citizen items. len: 4237884

In [11]:
statement_usa_citizen = selected_statements.loc[selected_statements["source_item_id"].isin(item_usa_citizen)]
print(len(statement_usa_citizen))

4237884


most frequent occupation in usa: 51175 82955 politician

In [12]:
cnt_occupation_usa = Counter(statement_usa_citizen.loc[statement_usa_citizen["edge_property_id"] == 106]["target_item_id"])
for k,v in cnt_occupation_usa.most_common(20):
    print(v, k, item2name(k))

51175 82955 politician
31894 33999 actor
25732 19204627 American football player
22616 40348 lawyer
22417 36180 writer
19989 10871364 baseball player
16828 10798782 television actor
16246 10800557 film actor
14279 1930187 journalist
12892 177220 singer
11489 36834 composer
11276 28389 screenwriter
10208 639669 musician
9869 3665646 basketball player
9481 16533 judge
8934 43845 businessperson
8204 3282637 film producer
8195 6625963 novelist
7680 1622272 university teacher
7450 2526255 film director


In [13]:
print(len(set(statement_usa_citizen["edge_property_id"])))
print(len(set(statement_usa_citizen["target_item_id"])))

268
205775


usa politician

In [14]:
item_usa_politician = statement_usa_citizen.loc[(statement_usa_citizen["edge_property_id"] == 106) & (statement_usa_citizen["target_item_id"] == 82955)]["source_item_id"]
print(len(item_usa_politician))
statement_usa_politician = df_statement.loc[df_statement["source_item_id"].isin(item_usa_politician)]
print(len(statement_usa_politician))
print(len(set(statement_usa_politician["edge_property_id"])))
print(len(set(statement_usa_politician["target_item_id"])))

51175
631821
155
42434


In [15]:
prop_usa_politician = set(statement_usa_politician["edge_property_id"])
print(len(prop_usa_politician))

155


1. create a prop-value matrix of human item.
2. compute similarities among them.

In [28]:
idx2item = {}
item2idx = {}
idx2prop = {}
prop2idx = {}

for index, item in enumerate(set(statement_usa_politician["source_item_id"])):
    idx2item[len(idx2item)] = item
for idx,item in idx2item.items():
    item2idx[item] = idx

for index, prop_id in enumerate(prop_usa_politician):
    idx2prop[index] = prop_id
for idx,prop in idx2prop.items():
    prop2idx[prop] = idx

In [29]:
print(len(idx2item))
print(len(item2idx))
print(len(idx2prop))
print(len(prop2idx))

51175
51175
155
155


In [50]:
import numpy as np
mat = np.zeros((len(idx2item), len(idx2prop)), dtype=int)
for index, row in statement_usa_politician.iterrows():   
    # print(item2idx[row["source_item_id"]], prop2idx[row["edge_property_id"]], row["target_item_id"]) 
    mat[item2idx[row["source_item_id"]]][prop2idx[row["edge_property_id"]]] = row["target_item_id"]

In [51]:
statement_usa_politician.iloc[0]

source_item_id          23
edge_property_id        19
target_item_id      494413
Name: 1233, dtype: int64

In [52]:
mat[item2idx[23]][prop2idx[19]]

494413

In [65]:
list_column = [v for x,v in sorted(idx2prop.items())]
print(len(list_column))
list_row = [v for x,v in sorted(idx2item.items())]
print(len(list_row))

155
51175


In [55]:
usa_politician_feature = pd.DataFrame(mat, columns=list_column)
usa_politician_feature.index = list_row

In [56]:
usa_politician_feature.iloc[item2idx[23], prop2idx[19]]

494413

In [57]:
usa_politician_feature.head()

,512,1026,5125,1037,1038,527,17,3602,19,20,...,452,463,2522,991,485,1000,5096,1532,509,511
0,0,0,0,0,0,0,0,0,38555,5911277,...,0,0,0,0,0,0,0,0,175111,0
1,0,0,0,0,0,0,0,0,967215,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12078,0
3,0,0,0,0,0,0,0,0,664723,65,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,34404,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
usa_politician_feature.to_csv(path+"feature_usa_politician.csv", index=True)

In [92]:
usa_politician_feature.loc[6294][19]

27787813

In [93]:
print(item2name(6294), prop2name(19), item2name(27787813))

Hillary Clinton place of birth Edgewater Hospital


In [74]:
df_item.loc[df_item["en_label"]=="Barack Obama"]

,item_id,en_label,en_description
65,76,Barack Obama,44th president of the United States
32370596,47513588,Barack Obama,painting by Michael A. Glier


In [76]:
df_item.loc[df_item["en_label"]=="Hillary Clinton"]

,item_id,en_label,en_description
5884,6294,Hillary Clinton,"American politician, senator, Secretary of Sta..."
32370270,47513201,Hillary Clinton,painting by Aaron Shikler


In [97]:
tmp_df = usa_politician_feature.loc[usa_politician_feature.index.isin([6294, 76])]


In [100]:
tmp_df

,512,1026,5125,1037,1038,527,17,3602,19,20,...,452,463,2522,991,485,1000,5096,1532,509,511
76,1765120,0,0,0,4115068,0,0,0,6366688,0,...,0,466089,0,0,0,0,0,0,0,0
6294,1540185,0,0,0,0,0,0,0,27787813,0,...,0,3075672,0,0,0,0,0,0,0,0


In [109]:
tmp_df

,512,1026,5125,1037,1038,527,17,3602,19,20,...,452,463,2522,991,485,1000,5096,1532,509,511
76,1765120,0,0,0,4115068,0,0,0,6366688,0,...,0,466089,0,0,0,0,0,0,0,0
6294,1540185,0,0,0,0,0,0,0,27787813,0,...,0,3075672,0,0,0,0,0,0,0,0


In [111]:
tmp_df1 = (tmp_df != 0).any()
tmp_df1

512      True
1026    False
5125    False
1037    False
1038     True
        ...  
1000    False
5096    False
1532    False
509     False
511     False
Length: 155, dtype: bool

In [104]:
prop2name(512)

'academic degree'

In [105]:
item2name(1540185)

'Juris Doctor'

In [106]:
item2name(1765120)

'Bachelor of Arts'

In [113]:
df_statement.head()

,source_item_id,edge_property_id,target_item_id
0,1,31,36906466
1,1,279,3695190
2,1,398,497745
3,1,398,1133705
4,1,398,1139177


In [115]:
df_item.loc[df_item["en_label"] == "University of Notre Dame"]

,item_id,en_label,en_description
166727,178848,University of Notre Dame,"Catholic university in South Bend, Indiana, Un..."
8441230,12778083,University of Notre Dame,Wikimedia disambiguation page


Take a look at statements of "University of Notre Dame"

In [127]:
df_nd = df_statement.loc[df_statement["source_item_id"]==178848]
for idx, row in df_nd.iterrows():
    print(row["edge_property_id"], row["target_item_id"], prop2name(row["edge_property_id"]), item2name(row["target_item_id"]))

17 30 country United States of America
31 902104 instance of private university
31 23002054 instance of private not-for-profit educational institution
112 2281954 founded by Edward Sorin
131 746071 located in the administrative territorial entity Notre Dame
140 9592 religion Catholic Church
355 45132619 subsidiary Kroc Institute for International Peace Studies,...
355 45135964 subsidiary Helen Kellogg Institute for International Studi...
355 55830228 subsidiary Institute for Structure and Nuclear Astrophysics
463 647039 member of Scholarly Publishing and Academic Resources Coa...
463 756355 member of Atlantic Coast Conference
463 4810036 member of Association of Research Libraries
463 5059850 member of Center for Research Libraries
463 5275906 member of Digital Library Federation
463 19861084 member of ORCID, Inc.
463 35677307 member of IIIF Consortium
488 6240609 chairperson John I. Jenkins
641 41323 sport American football
822 7063459 mascot Notre Dame Leprechaun
910 10085790 topic's

Now we know about "31 902104 instance of private university". Let's take a look at "University of Michigan"

In [128]:
df_item.loc[df_item["en_label"] == "University of Michigan"]

,item_id,en_label,en_description
207831,230492,University of Michigan,"public research university in Ann Arbor, Michi..."
13729669,20670914,University of Michigan,painting by Jasper Francis Cropsey


In [129]:
for idx, row in df_statement.loc[df_statement["source_item_id"]==230492].iterrows():
    print(row["edge_property_id"], row["target_item_id"], prop2name(row["edge_property_id"]), item2name(row["target_item_id"]))

17 30 country United States of America
31 23002039 instance of public educational institution of the United St...
31 62078547 instance of public research university
112 1489605 founded by Gabriel Richard
131 485172 located in the administrative territorial entity Ann Arbor
159 485172 headquarters location Ann Arbor
355 911830 subsidiary University of Michigan Law School
355 3304503 subsidiary University of Michigan College of Engineering
355 7895753 subsidiary University of Michigan College of Literature, S...
355 7895755 subsidiary University of Michigan College of Pharmacy
355 7895759 subsidiary University of Michigan Herbarium
355 7895772 subsidiary University of Michigan School of Dentistry
355 7895773 subsidiary University of Michigan School of Information
355 7895775 subsidiary University of Michigan School of Music, Theatre...
355 7895776 subsidiary University of Michigan School of Natural Resour...
355 7895778 subsidiary University of Michigan School of Public Health
355 169023

In [136]:
property_uni = df_item.loc[df_item["en_label"].str.contains("university", na=False)]["item_id"]

In [143]:
len(property_uni)

9490

In [141]:
dist_uni = Counter(df_statement.loc[(df_statement["target_item_id"].isin(property_uni))]["target_item_id"])

In [142]:
for k,v in dist_uni.most_common(30):
    print(item2name(k), k, v)

university teacher 1622272 31676
university 3918 13902
private university 902104 911
public university 875538 850
university and college sports club 2367225 593
university of applied sciences 1365560 352
university building 19844914 304
university hospital 1059324 269
university press 479716 240
university-preparatory school 428602 187
public research university 62078547 138
university museum 866133 122
doctoral university: highest research activity 23334765 115
land-grant university 615150 109
doctoral university: higher research activity 23334891 104
doctoral university: moderate research activity 23334993 99
research university 15936437 91
university in France 3551775 83
national university 265662 79
university college 7894996 70
Catholic university 557206 61
state university system 1075106 49
college and university rankings 847843 48
pontifical university 2120466 40
university college 15407956 36
Alborz university of medical sciences 5940463 36
Ural state university of physical cul

university 3918 13902

private university 902104 911

public university 875538 850

In [145]:
items_uni = df_statement.loc[(df_statement["edge_property_id"]==31) & (df_statement["target_item_id"].isin([3918, 902104, 875538]))]["source_item_id"]

In [146]:
statement_uni = df_statement.loc[df_statement["source_item_id"].isin(items_uni)]

In [147]:
statement_uni

,source_item_id,edge_property_id,target_item_id
38883,681,17,36
38884,681,31,875538
38885,681,31,45400320
38886,681,131,1280107
38887,681,463,868940
...,...,...,...
140543467,76737841,17,1029
140543468,76737841,31,875538
140665362,76822227,17,1049
140665363,76822227,31,3918


Next: 
- save the filtered statement.csv and the item.csv
- match Wikipedia id from item_id to page_id in page.csv
- get Wikipedia passages by page_id and save them